In [1]:
import lmdb
import pickle
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt

def inspect_lmdb(lmdb_path, num_samples=5):
    """
    检查 LMDB 数据集的内部结构。

    Args:
        lmdb_path (str): LMDB 数据集的路径。
        num_samples (int): 要查看的样本数量。
    """
    env = lmdb.open(lmdb_path, readonly=True, lock=False, readahead=False, meminit=False)
    with env.begin(write=False) as txn:
        cursor = txn.cursor()
        count = 0
        for key, value in cursor:
            unpacked = pickle.loads(value)
            img, label = unpacked

            print(f"样本 {count}:")
            print(f"键（Key）：{key.decode('utf-8')}")
            print(f"图像特征类型：{type(img)}")
            if isinstance(img, np.ndarray):
                print(f"图像特征形状：{img.shape}")
            elif isinstance(img, torch.Tensor):
                print(f"图像特征形状：{img.size()}")
            print(f"标签（Label）：{label}")
            print("-" * 50)

            # 如果图像特征可以转回图像进行可视化（取决于你的特征类型）
            # 这里假设 img 是图像的 NumPy 数组
            if isinstance(img, np.ndarray) and img.ndim == 3:
                # 反归一化（如果有必要）
                img_display = img.copy()
                if img_display.max() > 1.0:
                    img_display = img_display / 255.0
                plt.imshow(img_display)
                plt.title(f"Sample {count} Label: {label}")
                plt.show()

            count += 1
            if count >= num_samples:
                break

    env.close()

# 使用示例
inspect_lmdb('autodl-tmp/ek55/rgb/data.mdb', num_samples=3)


Error: autodl-tmp/ek55/rgb/data.mdb: No such file or directory